In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import spacy
import re
import nltk
from spacy.tokenizer import Tokenizer
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
from nltk import word_tokenize
from sklearn import utils

In [3]:
import csv
import re
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [4]:
dt = pd.read_csv('train.csv')

In [5]:
dt.dropna()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


# English

In [6]:
# english data set
df_en = dt.loc[dt['lang_abv'] == 'en']
df_en

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [7]:
x_p = pd.DataFrame()
x_p['p']=df_en['premise']
x_p['label']= df_en['label']
x_h = pd.DataFrame()
x_h['p']=df_en['hypothesis']
x_h['label']= df_en['label']
# split train and test
X_train_p, X_test_p, X_train_h, X_test_h= train_test_split(x_p,x_h, test_size=0.3, random_state=100)

In [8]:
#create a csv file
X_train_p.to_csv('X_train_p.csv', encoding='utf-8', index = False)
X_test_p.to_csv('X_test_p.csv', encoding='utf-8', index = False)
X_train_h.to_csv('X_train_h.csv', encoding='utf-8', index = False)
X_test_h.to_csv('X_test_h.csv', encoding='utf-8', index = False)

In [9]:
#load the csv file to create the input data for Doc2vec
data_train_p = list()
with open('X_train_p.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        data_train_p.append(TaggedDocument(word_tokenize(row[0].lower()),[row[1]]))
        
#load the csv file to create the input data for Doc2vec
data_train_h = list()
with open('X_train_h.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        data_train_h.append(TaggedDocument(word_tokenize(row[0].lower()),[row[1]]))

In [10]:
data_train_p.pop(0)
data_train_h.pop(0)

TaggedDocument(words=['p'], tags=['label'])

In [11]:
#load the csv file to create the input data for Doc2vec
data_test_p = list()
with open('X_train_p.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        data_test_p.append(TaggedDocument(word_tokenize(row[0].lower()),[row[1]]))
        
#load the csv file to create the input data for Doc2vec
data_test_h = list()
with open('X_train_p.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        data_test_h.append(TaggedDocument(word_tokenize(row[0].lower()),[row[1]]))

In [12]:
data_test_p.pop(0)
data_test_h.pop(0)

TaggedDocument(words=['p'], tags=['label'])

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [14]:
import multiprocessing
import tqdm
cores = multiprocessing.cpu_count()

In [15]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab(data_train_p)

2022-09-29 16:32:13,896 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d300,n5,mc2,t8)', 'datetime': '2022-09-29T16:32:13.896520', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-09-29 16:32:13,899 : INFO : collecting all words and their counts
2022-09-29 16:32:13,899 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-09-29 16:32:13,930 : INFO : collected 10080 word types and 3 unique tags from a corpus of 4809 examples and 104039 words
2022-09-29 16:32:13,932 : INFO : Creating a fresh vocabulary
2022-09-29 16:32:13,968 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 7418 unique words (73.59126984126983%% of original 10080, drops 2662)', 'datetime': '2022-09-29T16:32:13.968226', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.

In [17]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle(data_train_p), total_examples=len(data_train_p), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

2022-09-29 16:32:46,934 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 7418 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-29T16:32:46.934684', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:47,121 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-29 16:32:47,129 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-29 16:32:47,135 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-29 16:32:47,144 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-29 16:32:47,146 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-29 16:32:47,161 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-29 16:32:47,192 : INFO : worker thread finished; awaiting finish o

2022-09-29 16:32:48,428 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-29 16:32:48,430 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-29 16:32:48,433 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-29 16:32:48,442 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-29 16:32:48,446 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-29 16:32:48,466 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-29 16:32:48,497 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-29 16:32:48,497 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-29 16:32:48,498 : INFO : EPOCH - 1 : training on 104039 raw words (106186 effective words) took 0.3s, 423592 effective words/s
2022-09-29 16:32:48,498 : INFO : Doc2Vec lifecycle event {'msg': 'training on 104039 raw words (106186 effective words) took 0.3s, 406868

2022-09-29 16:32:49,640 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-29 16:32:49,641 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-29 16:32:49,662 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-29 16:32:49,693 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-29 16:32:49,697 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-29 16:32:49,698 : INFO : EPOCH - 1 : training on 104039 raw words (106186 effective words) took 0.2s, 450120 effective words/s
2022-09-29 16:32:49,699 : INFO : Doc2Vec lifecycle event {'msg': 'training on 104039 raw words (106186 effective words) took 0.2s, 430978 effective words/s', 'datetime': '2022-09-29T16:32:49.699140', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:49,700 : INFO : Doc2Vec lifecycle e

2022-09-29 16:32:50,869 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-29 16:32:50,872 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-29 16:32:50,873 : INFO : EPOCH - 1 : training on 104039 raw words (106186 effective words) took 0.2s, 501695 effective words/s
2022-09-29 16:32:50,874 : INFO : Doc2Vec lifecycle event {'msg': 'training on 104039 raw words (106186 effective words) took 0.2s, 477052 effective words/s', 'datetime': '2022-09-29T16:32:50.874497', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:50,875 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 7418 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-29T16:32:50.875494', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'pla

2022-09-29 16:32:52,079 : INFO : Doc2Vec lifecycle event {'msg': 'training on 104039 raw words (106186 effective words) took 0.2s, 448031 effective words/s', 'datetime': '2022-09-29T16:32:52.079053', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:52,081 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 7418 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-29T16:32:52.081009', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:52,259 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-29 16:32:52,263 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-29 16:32:52,265 : INFO : worker thread finished; awaiting finish of 5 mo

2022-09-29 16:32:53,260 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 7418 vocabulary and 300 features, using sg=1 hs=0 sample=0 negative=5 window=5', 'datetime': '2022-09-29T16:32:53.260161', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}
2022-09-29 16:32:53,431 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-29 16:32:53,435 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-29 16:32:53,436 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-29 16:32:53,443 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-29 16:32:53,449 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-29 16:32:53,461 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-29 16:32:53,486 : INFO : worker thread finished; awaiting finish o

Wall time: 7.25 s


In [18]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [19]:
y_train, X_train_p = vec_for_learning(model_dbow, data_train_p)
y_test, X_test_p = vec_for_learning(model_dbow, data_test_p)
y_train, X_train_h = vec_for_learning(model_dbow, data_train_h)
y_test, X_test_h = vec_for_learning(model_dbow, data_test_h)

In [20]:
a = np.hstack((X_train_p , X_train_h))
b = np.hstack((X_test_p , X_test_h)) 

In [21]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(a)
X_train = scaler.transform(a)
X_test = scaler.transform(b)

## Naive Bayes Classifier

In [22]:
model = MultinomialNB()

model.fit(X_train,y_train)
predictions = model.predict(X_test)

print('Classification report:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.38      0.10      0.15      1724
           1       0.32      0.67      0.43      1499
           2       0.34      0.26      0.30      1586

    accuracy                           0.33      4809
   macro avg       0.35      0.34      0.30      4809
weighted avg       0.35      0.33      0.29      4809

Confusion matrix:
[[ 165 1125  434]
 [ 123 1010  366]
 [ 142 1025  419]]


## Linear SVM

In [23]:
# without pipeline
clf = LinearSVC(C=1.0, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.39904346017883136
F1-score [0.4517556  0.44217321 0.24529128]
              precision    recall  f1-score   support

           0       0.43      0.47      0.45      1724
           1       0.37      0.55      0.44      1499
           2       0.40      0.18      0.25      1586

    accuracy                           0.40      4809
   macro avg       0.40      0.40      0.38      4809
weighted avg       0.40      0.40      0.38      4809



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [25]:
SVM_opt_search.best_params_

{'learner__C': 1, 'sel__k': 'all'}

In [26]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.46      0.41      0.43      1724
           1       0.36      0.62      0.46      1499
           2       0.39      0.18      0.24      1586

    accuracy                           0.40      4809
   macro avg       0.41      0.40      0.38      4809
weighted avg       0.41      0.40      0.38      4809

Confusion matrix:
[[703 808 213]
 [344 934 221]
 [483 823 280]]


In [30]:
from sklearn.svm import SVC

In [31]:
clf = clf = SVC(gamma='auto', C=0.1, kernel='rbf', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.3584944894988563
F1-score [0.52778203 0.         0.        ]
              precision    recall  f1-score   support

           0       0.36      1.00      0.53      1724
           1       0.00      0.00      0.00      1499
           2       0.00      0.00      0.00      1586

    accuracy                           0.36      4809
   macro avg       0.12      0.33      0.18      4809
weighted avg       0.13      0.36      0.19      4809



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
clf.get_params()

{'C': 0.1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [29]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', SVC(kernel= 'rbf'))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [30]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.31      0.49      0.38       703
           1       0.27      0.15      0.19       667
           2       0.28      0.24      0.26       691

    accuracy                           0.29      2061
   macro avg       0.29      0.29      0.28      2061
weighted avg       0.29      0.29      0.28      2061

Confusion matrix:
[[345 129 229]
 [385  98 184]
 [389 138 164]]


## Random Forest

In [27]:
rd =  RandomForestClassifier(bootstrap = True)
rd.fit(X_train,y_train)
pred = rd.predict(X_test)

print('Classification report:')
print(classification_report(y_test, pred, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, pred)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.40      0.43      0.42      1724
           1       0.35      0.54      0.43      1499
           2       0.39      0.17      0.24      1586

    accuracy                           0.38      4809
   macro avg       0.38      0.38      0.36      4809
weighted avg       0.38      0.38      0.36      4809

Confusion matrix:
[[744 745 235]
 [507 803 189]
 [596 718 272]]


In [28]:
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__n_estimators': [200, 300, 500, 700]}]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [29]:
opt_search.best_params_

{'learner__n_estimators': 500, 'sel__k': 'all'}

In [30]:
opt_predictions = opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.41      0.51      0.45      1724
           1       0.36      0.48      0.41      1499
           2       0.36      0.15      0.21      1586

    accuracy                           0.38      4809
   macro avg       0.38      0.38      0.36      4809
weighted avg       0.38      0.38      0.36      4809

Confusion matrix:
[[872 629 223]
 [573 723 203]
 [679 669 238]]
